# OpenAI Api


## 初始化


### 初始化环境变量


In [ ]:
import { config } from "dotenv";

await config({ export: true });

const apiKey = Deno.env.get("API_KEY");
const baseURL = Deno.env.get("BASE_URL");
const systemPrompt = Deno.env.get("SYSTEM_PROMPT");


### 初始化 ai 实例


In [2]:
import OpenAI from "openai";
const openai = new OpenAI({
  apiKey,
  baseURL,
});


## 调用 chat.completion 方法


In [ ]:
const result = openai.chat.completions.create({
  model: "gpt-4o",
  stream: true,
  temperature: 0.4,
  messages: [
    {
      role: "system",
      content: systemPrompt,
    },
    {
      role: "user",
      content: "生成一个登陆界面",
    },
  ],
});

let assistantContent = "";

for await (const chunk of await result) {
  console.log(chunk?.choices?.[0]?.delta?.content || "");
  assistantContent += chunk?.choices?.[0]?.delta?.content || "";
}

console.log(assistantContent);


## 调用 function calling


In [ ]:
const result = openai.chat.completions.create({
  model: "gpt-4o",
  stream: true,
  temperature: 0.4,
  messages: [
    {
      role: "system",
      content: "提取出用户提供的代码中用到了哪些antd的基础组件",
    },
    {
      role: "user",
      content: assistantContent,
    },
  ],
  tools: [
    {
      type: "function",
      function: {
        name: `extract_antd_components`,
        description: `提取出用户提供的代码中用到了哪些antd的基础组件`,
        parameters: {
          type: "object",
          properties: {
            components: {
              type: "array",
              items: {
                type: "string",
              },
            },
          },
          required: ["components"],
        },
      },
    },
  ],
});
let functionCallingContent = "";
for await (const chunk of await result) {
  console.log(
    chunk?.choices?.[0]?.delta?.tool_calls?.[0]?.function?.arguments || ""
  );
  functionCallingContent +=
    chunk?.choices?.[0]?.delta?.tool_calls?.[0]?.function?.arguments || "";
}

console.log(functionCallingContent);


## json_object 输出模式


In [ ]:
const result = openai.chat.completions.create({
  model: "gpt-4o",
  stream: true,
  temperature: 0.4,
  messages: [
    {
      role: "system",
      content: "帮我提取出用户提供的代码中用到了哪些antd的基础组件，输出为json",
    },
    {
      role: "user",
      content: assistantContent,
    },
  ],
  response_format: {
    type: "json_object",
  },
});

let jsonObjectContent = "";

for await (const chunk of await result) {
  console.log(chunk?.choices?.[0]?.delta?.content || "");
  jsonObjectContent += chunk?.choices?.[0]?.delta?.content || "";
}

console.log(jsonObjectContent);


## Structured Outputs


In [ ]:
import { zodResponseFormat } from "openai/helpers/zod";
import { z } from "zod";

const componentsResponseSchema = z.object({
  components: z.array(z.string()),
});


const result = await openai.beta.chat.completions.parse({
  model: "gpt-4o",
  temperature: 0.4,
  messages: [
    {
      role: "system",
      content: "帮我提取出用户提供的代码中用到了哪些antd的基础组件",
    },
    {
      role: "user",
      content: assistantContent,
    },
  ],
  response_format: zodResponseFormat(componentsResponseSchema, "event"),
});

const event = result.choices[0].message.parsed;

console.log(event);


> OpenAI 更多资料详见官方文档：https://platform.openai.com/docs/overview